In [ ]:
# Import Libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Uncomment if using google drive and google colab

# from google.colab import drive
# drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Uncomment if using google drive and google colab

# !unzip /content/gdrive/MyDrive/archive.zip -d archive

Streaming output truncated to the last 5000 lines.
  inflating: archive/train/jack of spades/059.jpg  
  inflating: archive/train/jack of spades/060.jpg  
  inflating: archive/train/jack of spades/061.jpg  
  inflating: archive/train/jack of spades/062.jpg  
  inflating: archive/train/jack of spades/063.jpg  
  inflating: archive/train/jack of spades/064.jpg  
  inflating: archive/train/jack of spades/065.jpg  
  inflating: archive/train/jack of spades/066.jpg  
  inflating: archive/train/jack of spades/067.jpg  
  inflating: archive/train/jack of spades/068.jpg  
  inflating: archive/train/jack of spades/069.jpg  
  inflating: archive/train/jack of spades/070.jpg  
  inflating: archive/train/jack of spades/071.jpg  
  inflating: archive/train/jack of spades/072.jpg  
  inflating: archive/train/jack of spades/073.jpg  
  inflating: archive/train/jack of spades/074.jpg  
  inflating: archive/train/jack of spades/075.jpg  
  inflating: archive/train/jack of spades/076.jpg  
  inflating: 

In [ ]:
train_ds = image_dataset_from_directory(
    directory='train',
    labels='inferred',
    label_mode='categorical',
    batch_size=128,
    image_size=(224, 224))
validation_ds = image_dataset_from_directory(
    directory='valid',
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(224,224))

Found 7624 files belonging to 53 classes.
Found 265 files belonging to 53 classes.


In [ ]:
# This model uses transfer learning from ResNet101

model = Sequential()

resnet_model = ResNet101(
    include_top=False,
    input_shape=(224,224,3),
    pooling='max',
    classes=53,
    weights='imagenet'
)

for layer in resnet_model.layers:
    layer.trainable=False
    
model.add(resnet_model)

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(159, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(106, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(53, activation='softmax'))

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

In [ ]:
# Train the transfer learning model

epochs=100
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=epochs
)

Epoch 1/100
60/60 [==============================] - 60s 747ms/step - loss: 4.5607 - accuracy: 0.0404 - val_loss: 3.7021 - val_accuracy: 0.0868
Epoch 2/100
60/60 [==============================] - 43s 703ms/step - loss: 3.5750 - accuracy: 0.0877 - val_loss: 2.9382 - val_accuracy: 0.1547
Epoch 3/100
60/60 [==============================] - 42s 690ms/step - loss: 3.1857 - accuracy: 0.1213 - val_loss: 2.4846 - val_accuracy: 0.2453
Epoch 4/100
60/60 [==============================] - 43s 700ms/step - loss: 2.9394 - accuracy: 0.1592 - val_loss: 2.3164 - val_accuracy: 0.2415
Epoch 5/100
60/60 [==============================] - 43s 700ms/step - loss: 2.7702 - accuracy: 0.1881 - val_loss: 2.1383 - val_accuracy: 0.3245
Epoch 6/100
60/60 [==============================] - 42s 691ms/step - loss: 2.6505 - accuracy: 0.2184 - val_loss: 2.0386 - val_accuracy: 0.3547
Epoch 7/100
60/60 [==============================] - 42s 691ms/step - loss: 2.5573 - accuracy: 0.2299 - val_loss: 1.9823 - val_accuracy:

In [ ]:
# This second model is made from 'scratch' using tensorflow and keras

scratch_model = Sequential()

scratch_model.add(Conv2D(32,kernel_size=6, activation='relu', input_shape=(224,224,3)))
scratch_model.add(BatchNormalization())
scratch_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

scratch_model.add(Conv2D(32, kernel_size=3, activation='relu'))
scratch_model.add(BatchNormalization())
scratch_model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2)))

scratch_model.add(Conv2D(32, kernel_size=3, activation='relu'))
scratch_model.add(BatchNormalization())
scratch_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

scratch_model.add(Conv2D(32, kernel_size=3, activation='relu'))
scratch_model.add(BatchNormalization())
scratch_model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2)))

scratch_model.add(Conv2D(32, kernel_size=3, activation='relu'))
scratch_model.add(BatchNormalization())
scratch_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

scratch_model.add(Flatten())


scratch_model.add(Dropout(0.2))
scratch_model.add(Dense(159, activation='relu'))
scratch_model.add(Dropout(0.2))
scratch_model.add(Dense(106, activation='relu'))
scratch_model.add(Dropout(0.2))
scratch_model.add(Dense(53, activation='softmax'))

scratch_model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

scratch_model.summary()

In [ ]:
# Train the second model
# This model should reach about 83 percent accuracy on validation data before overfitting

epochs=100
history = scratch_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=epochs
)

Epoch 1/100
60/60 [==============================] - 19s 282ms/step - loss: 3.6820 - accuracy: 0.0959 - val_loss: 3.4963 - val_accuracy: 0.1358
Epoch 2/100
60/60 [==============================] - 18s 286ms/step - loss: 2.7374 - accuracy: 0.2419 - val_loss: 2.3025 - val_accuracy: 0.3660
Epoch 3/100
60/60 [==============================] - 18s 284ms/step - loss: 2.3730 - accuracy: 0.3103 - val_loss: 2.4445 - val_accuracy: 0.2868
Epoch 4/100
60/60 [==============================] - 18s 289ms/step - loss: 2.1620 - accuracy: 0.3597 - val_loss: 2.4403 - val_accuracy: 0.3321
Epoch 5/100
60/60 [==============================] - 17s 278ms/step - loss: 1.9679 - accuracy: 0.4359 - val_loss: 1.8256 - val_accuracy: 0.4830
Epoch 6/100
60/60 [==============================] - 18s 280ms/step - loss: 1.7869 - accuracy: 0.4903 - val_loss: 1.7469 - val_accuracy: 0.5585
Epoch 7/100
60/60 [==============================] - 18s 280ms/step - loss: 1.6264 - accuracy: 0.5354 - val_loss: 1.1853 - val_accuracy: